#✅ Bootcamp: Automating Sports Journalism with AI & Data

##📌 Module Overview
This module teaches participants how to automatically generate written sports news articles using structured data and large language models. Students will write Python code that takes player stats, game summaries, or match events from a CSV/Excel file and turns them into full paragraphs of readable, journalistic content.

### Example used

![image](https://www.alliancefencingequipment.com/media/product/picture/2019/06/14/alliance.jpg)

## ✅ Step 1: Introduction to AI-Driven Sports Journalism

**Goal**: Understand the problem space and real-world value.

###Topics:

- What is data-driven sports journalism?
- Real-world use cases (e.g. ESPN, Opta, The Athletic)
- Why LLMs + structured data are powerful together

##📋Pre-Requisite

- Download Sports data from [Score Sports Data Repository](https://data.scorenetwork.org/)
- Or Use scraper to scrape the data



##✅ Step 2: Install Dependencies and Load Data

**Goal**: Set up Python environment and load sample data.

###Topics:

- Using pandas to read CSV or Excel
- Inspecting sports match/player data

In [1]:
from google.colab import files

uploaded = files.upload()

Saving NCAAfencing.csv to NCAAfencing.csv


In [2]:
import pandas as pd

# Load a sample cricket or football match stats file
df = pd.read_csv("NCAAfencing.csv")
df.head()

# ✅ Make sure data includes columns like player name, team, goals/runs, minutes, etc.

Place   Tied             Name      School  Victories  Bouts    Pct   TS  \
0      1  False    Maggie Shealy    Brandeis         18     23  0.783  109   
1      2  False    Julia Cieslar  St. John's         14     23  0.609   92   
2      3   True     Ryan Jenkins   Princeton         18     23  0.783  102   
3      3   True        Vera Kong     Barnard         16     23  0.696   98   
4      5  False  Atara Greenbaum  Notre Dame         14     23  0.609   93   

   TR  Ind Gender Weapon      Ivy  
0  74   35  Women  Sabre  Non-Ivy  
1  75   17  Women  Sabre  Non-Ivy  
2  62   40  Women  Sabre      Ivy  
3  67   31  Women  Sabre      Ivy  
4  79   14  Women  Sabre  Non-Ivy

##✅ Step 3: Prepare a Prompt Template

**Goal**: Create a reusable text template to convert stats into prompts for GPT.

###Topics:
- Prompt engineering basics
- Using f-strings with Pandas row data

In [3]:
def create_prompt(row):
    return (
        f"Write a 3–4 sentence sports article summary about a fencing athlete based on the following stats:\n"
        f"Name: {row['Name']}, School: {row['School']}, Gender: {row['Gender']}, Weapon: {row['Weapon']}, "
        f"Victories: {row['Victories']} out of {row['Bouts']} bouts (Win Rate: {row['Pct']:.3f}).\n"
        f"Touches Scored (TS): {row['TS']}, Touches Received (TR): {row['TR']}, Indicator: {row['Ind']}.\n"
        f"This athlete is from an {'Ivy League' if row['Ivy'] == 'Ivy' else 'Non-Ivy'} school.\n"
        f"Write it in an energetic, competitive tone like a post-tournament recap."
    )

##✅ Step 4: Generate AI Articles Using OpenAI API

**Goal**: Use OpenAI to generate text summaries.

###Topics:
- Connecting to OpenAI API
- Looping over data rows
- Collecting AI-generated outputs

In [4]:
import openai

# Use your deployed model name
deployment_name = "gpt-4o" # This is now passed as 'model'

# Initialize the OpenAI client with Azure parameters
# Make sure to replace the placeholder values with your actual Azure details
client = openai.AzureOpenAI(
    azure_endpoint = "https://chand-m2kbnjta-southcentralus.openai.azure.com/",
    api_key="",
    api_version="2023-05-15" # Use the API version supported by your deployment
)


def generate_article(prompt):
    # Use the new client.chat.completions.create method
    response = client.chat.completions.create(
        model=deployment_name,  # This is your Azure deployment name passed as model
        messages=[{"role": "user", "content": prompt}]
    )
    # Access the content from the response object differently in v1.0.0+
    return response.choices[0].message.content

df["article"] = df.apply(lambda row: generate_article(create_prompt(row)), axis=1)

##✅ Step 5: Save and Export the Articles in Markdown Format

**Goal**: Export each AI-generated article into a separate .md file using the player's name and match index as the filename.

###Topics:
- Writing Markdown files in Python
- Formatting titles and content
- Avoiding filename issues (e.g., spaces, special characters)

In [ ]:
import os

# Create output folder if it doesn't exist
os.makedirs("articles_md", exist_ok=True)

# Save each article as a Markdown file
for idx, row in df.iterrows():
    player_name = row['Name'].replace(" ", "_").lower()
    school_name = row['School']
    filename = f"articles_md/{idx+1}_{player_name}.md"

    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"# Match Recap: {row['Name']} ({school_name})\n\n")
        f.write(row["article"])

In [ ]:
from google.colab import files
files.download("articles_md/1_maggie_shealy.md")  # Replace with your actual filename

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📂 This will generate markdown files like:

- articles_md/1_tom_courant.md

- articles_md/2_jason_roy.md

Each file will have a clean structure with a Markdown header (`# Match Recap: …`) followed by the AI-generated content.